In [1]:
# Download the YOLO-V5 repository.
!git clone https://github.com/ultralytics/yolov5.git

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [2]:
%cd yolov5

# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

/Users/xz/Desktop/Deep_Learning/finalproject/yolov5
Setup complete. Using torch 1.13.1 CPU


In [3]:
!ls

CITATION.cff     benchmarks.py    hubconf.py       train.py
CONTRIBUTING.md  classify         models           tutorial.ipynb
LICENSE          data             requirements.txt utils
README.md        detect.py        runs             val.py
README.zh-CN.md  dog-breeds-2     segment          wandb
__pycache__      export.py        setup.cfg


In [4]:
# Install beautyful soup library.
%pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Import required libraries to process the label data.
from glob import glob
from bs4 import BeautifulSoup
import os

In [6]:
ann = "../Stanford_Dogs_Dataset/annotations/Annotation"
for folder in os.listdir(ann):
    par = os.path.join(ann, folder)
    if ".DS" not in folder:
        for file in os.listdir(os.path.join(ann, folder)):
            if (".xml" not in file):
                os.rename(os.path.join(par, file), os.path.join(par, file + ".xml"))

In [7]:
# Create a label list of label xmal files. 
labels_path = "../Stanford_Dogs_Dataset/annotations/Annotation"
labels_dirs = os.listdir(labels_path)
label_s_dir = []
# print(labels_dirs)
for label in labels_dirs:
    label_dir = os.path.join(labels_path, label)
    labels_dir = list(sorted(glob(label_dir + "/*.xml")))

In [8]:
# Create dataset->export->labels foler and a list of txt lables.
text_labels_dir = '../dataset/export/labels'
os.makedirs(text_labels_dir, exist_ok = True)

In [9]:
for label in labels_dirs:
    label_dir = os.path.join(labels_path, label)
    labels_dir = list(sorted(glob(label_dir + "/*.xml")))
#     print(labels_dir)
    # Change labels from xml to txt.
    for labels in labels_dir : 
        with open(labels, 'r') as f :
            data = f.read()
            soup = BeautifulSoup(data, 'xml')

            img_size = soup.find('size')
            img_width = int(img_size.find('width').text)
            img_height = int(img_size.find('height').text)

            objects = soup.find_all('object')
            obj_list = []
            class_lambda = lambda x : 0 if x == 'cat' else 1
            for obj in objects :
                label = class_lambda(obj.find('name').text)
                xmin = int(obj.find('xmin').text)
                ymin = int(obj.find('ymin').text)
                xmax = int(obj.find('xmax').text)
                ymax = int(obj.find('ymax').text)

                x = ((xmin + xmax) / 2) / img_width
                y = ((ymin + ymax) / 2) / img_height
                width = (xmax - xmin) / img_width
                height = (ymax - ymin) / img_height

                obj_list.append([label, x, y, width, height])

            labels = labels.split("/")[-1]
            txt_label_dir = os.path.join(text_labels_dir,labels[:-4] + '.txt')

            with open(txt_label_dir, 'w') as f :
                for obj in obj_list :
                    f.write(str(obj[0]) + ' ' +\
                            str(obj[1]) + ' ' +\
                            str(obj[2]) + ' ' +\
                            str(obj[3]) + ' ' +\
                            str(obj[4]))

In [10]:
# Import required library to process the images.
from distutils.dir_util import copy_tree

In [11]:
# Create a images directory under the export directory.
imgs_dir = '../dataset/export/images'
os.makedirs(imgs_dir, exist_ok = True)

In [12]:
!find . -name "*.DS_Store" -type f -delete

In [13]:
# Copy the iamges to the new directory.
path = "../Stanford_Dogs_Dataset/images/Images";
dirs = os.listdir(path)
for file in dirs:
    if file != ".DS_Store":
        pic_dir = os.path.join(path, file)
        copy_tree(pic_dir,imgs_dir)

In [14]:
# Import the spliter.
from sklearn.model_selection import train_test_split

In [15]:
# Splite the image dataset for traning and validation.
img_list = glob(imgs_dir + '/*.jpg')
train_img, valid_img = train_test_split(img_list, test_size = 0.1, random_state = 0)

In [16]:
# Save training and validation image list.
with open('../dataset/train.txt', 'w') as f:
    f.write('\n'.join(train_img) + '\n')
with open('../dataset/val.txt', 'w') as f:
    f.write('\n'.join(valid_img) + '\n')

In [17]:
# Import yaml library to prepare configuration.
import yaml

In [18]:
# Define the yaml file for training.
with open('../dataset/data.yaml', 'w') as f:
    data = {
        'train' : '../dataset/train.txt',
        'val' : '../dataset/val.txt',
        'nc' : 120,
        'names' : dirs
    }
    yaml.dump(data, f)

In [19]:
!python ../yolov5/train.py \
--img 224 \
--batch 32 \
--epochs 100 \
--data ../dataset/data.yaml \
--cfg ./models/yolov5s.yaml \
--weights yolov5s.pt \
--name dog_yolov5s_results

# Finish it in 100 epoches

train: weights=yolov5s.pt, cfg=./models/yolov5s.yaml, data=../dataset/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=32, imgsz=224, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=dog_yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-46-g96a71b1 Python-3.9.13 torch-1.13.1 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=

In [ ]:
# If get dataset from Roboflow instead of local dataset

In [6]:
# Get your download code from from Roboflow
!pip install -q roboflow


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="ZnpCjOPPvd3ekugIaChf");
project = rf.workspace("class-dtdbh").project("dog-breeds-wvthe");
dataset = project.version(2).download("yolov5");

In [ ]:
!ls

In [ ]:
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
%cat {dataset.location}/data.yaml

In [ ]:
# set up environment
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

In [ ]:
# define number of classes based on YAML
import yaml
with open("yolov5/dog_breeds_dataset/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
!ls

In [ ]:
#this is the model configuration we will use
%cat models/yolov5s.yaml

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate dog_breeds_dataset/dog_breeds_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

Here, we are able to pass a number of arguments:

img: define input image size
batch: determine batch size
epochs: define the number of training epochs.
data: Our dataset locaiton is saved in the dataset.location
weights: specify a path to weights to start transfer learning from. 
cache: cache images for faster training

In [ ]:
%%time
# train yolov5s on dog breed data for 100 epochs
# time its performance
!python train.py --img 224 --batch 32 --epochs 100 --data dog_breeds_dataset/data.yaml --cfg dog_breeds_dataset/dog_breeds_yolov5s.yaml --weights '' --name yolov5s_dog_breeds_results  --cache

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
!python detect.py --weights runs/train/exp/weights/best.pt --img 416 --conf 0.1 --source {dataset.location}/test/images

In [ ]:
#display inference on ALL test images

import glob
from IPython.display import Image, display

for imageName in glob.glob('runs/detect/exp/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")
    break